In [13]:
from langchain_community.document_loaders import PyPDFLoader,PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import numpy as np
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


In [14]:
loader=PyPDFDirectoryLoader('./us_sensus')
documents=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
texts=text_splitter.split_documents(documents)
texts[0].page_content

'Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium \nsubsidies for those with incomes above 400 percent \nof the poverty level as well as for unemployed people.\n3\nIn addition to national policies, individual states and \nthe District of Columbia can affect health insurance \ncoverage by making Marketplace or Medicaid more'

In [15]:
len(texts)

316

In [16]:
hf_embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device':'cpu'},encode_kwargs={'normalize_embeddings':True})

C:\Users\ARSH\AppData\Local\Temp\ipykernel_2732\3404273392.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device':'cpu'},encode_kwargs={'normalize_embeddings':True})


In [17]:
import numpy as np
np.array(hf_embeddings.embed_query(texts[0].page_content))

array([-2.09632944e-02, -5.39067667e-04,  5.50213419e-02,  3.04131806e-02,
        6.77258000e-02,  1.14703082e-01, -2.38763355e-02, -9.44041088e-03,
       -9.48234797e-02, -5.78315090e-03, -1.78809818e-02,  9.28241313e-02,
       -3.16224135e-02, -1.03725754e-01,  1.14526181e-02,  9.99598671e-03,
        2.77782101e-02, -3.16832364e-02, -2.14311052e-02,  9.49425325e-02,
       -4.78349775e-02,  2.48226654e-02, -5.71239144e-02, -3.00199469e-03,
        8.33480991e-03,  2.94881710e-03,  3.43687050e-02, -2.18156073e-02,
       -1.04256282e-02,  6.41097277e-02,  1.02224529e-01, -1.80637632e-02,
        2.31494512e-02,  2.37538759e-02,  2.47497950e-02, -4.93023917e-02,
       -4.57996204e-02,  2.13811081e-02, -9.22349617e-02,  2.33677477e-02,
       -3.34298462e-02, -3.14886384e-02, -8.72596055e-02,  1.25024751e-01,
        1.99524667e-02,  1.27728982e-03, -4.58287634e-02,  8.62977877e-02,
        7.01130880e-03,  6.47120550e-02,  1.39410542e-02,  1.68929156e-02,
        4.69882935e-02,  

In [18]:
## VectorStore Creation
vectorstore=FAISS.from_documents(texts[:120],hf_embeddings)

In [ ]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_documents=vectorstore.similarity_search(query)

print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [20]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000234A5A83010> search_kwargs={'k': 3}


In [ ]:
# Load API key securely from environment variables
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# Warning if API key not found
if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    print("⚠️ Warning: HUGGINGFACEHUB_API_TOKEN not found in environment variables!")
    print("Please add your HuggingFace API key to your .env file")

In [56]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={
        "temperature": 0.01,  # Changed from 0 to 0.01 (small positive value)
        "max_new_tokens": 300
    }
)

llm = hf 
llm.invoke(query)

Device set to use cpu


'WHAT IS THE AMERICAN COMMUNITY SURVEY?\n\nThe American Community Survey (ACS) is a nationally representative survey of American adults conducted by the American Community Survey (ACS). The ACS is a nationally representative survey of American adults conducted by the American Community Survey (ACS). The ACS is a nationally representative survey of American adults conducted by the American Community Survey (ACS).\n\nWhat is the American Community Survey?\n\nThe American Community Survey (ACS) is a nationally representative survey of American adults conducted by the American Community Survey (ACS). The ACS is a nationally representative survey of American adults conducted by the American Community Survey (ACS).\n\nWhat is the American Community Survey (ACS) is a nationally representative survey of American adults conducted by the American Community Survey (ACS). The ACS is a nationally representative survey of American adults conducted by the American Community Survey (ACS).\n\nWhat is t

In [52]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [53]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [54]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [57]:
query="""WHAT IS HEALTH INSURANCE COVERAGE?"""

In [58]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased direct